In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, datasets, utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import numpy as np
import cv2

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callback per salvare il modello con la migliore accuratezza di validazione
checkpoint_cb = ModelCheckpoint("best_model.h5", save_best_only=True, monitor='val_accuracy')

# Callback per fermare l'addestramento se non c'è miglioramento in 'patience' epoche
early_stopping_cb = EarlyStopping(patience=10, restore_best_weights=True, monitor='val_accuracy')

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import datasets, utils

# Load Fashion MNIST data
(x_train, y_train), (x_test, y_test) = datasets.fashion_mnist.load_data()

# Upscale images from 28x28 to 32x32 and add 3 channels
x_train = np.pad(x_train, ((0,0), (2,2), (2,2)), 'constant', constant_values=0)
x_test = np.pad(x_test, ((0,0), (2,2), (2,2)), 'constant', constant_values=0)
x_train = np.stack((x_train,)*3, axis=-1)
x_test = np.stack((x_test,)*3, axis=-1)

# Normalize data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# Convert labels to one-hot encoding
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

input_shape = x_train.shape[1:]  # Now (32, 32, 3)


In [ ]:
input_shape

(32, 32, 3)

### resnet + adapters

Step 1: Select a Pre-trained Model

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model

# Load the base ResNet50 model without the top layer
base_resnet_model = ResNet50(include_top=False, weights=None, input_shape=(32, 32, 3))

# Add new top layers
x = base_resnet_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(10, activation='softmax')(x)

# This is the model we will train
base_model = Model(inputs=base_resnet_model.input, outputs=predictions)

# Compile the model
base_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history_base_model = base_model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 125s 47ms/step - loss: 0.7728 - accuracy: 0.7614 - val_loss: 0.5970 - val_accuracy: 0.7856
Epoch 2/10
1875/1875 [==============================] - 86s 46ms/step - loss: 0.5700 - accuracy: 0.8231 - val_loss: 0.6966 - val_accuracy: 0.7570
Epoch 3/10
1875/1875 [==============================] - 84s 45ms/step - loss: 0.6006 - accuracy: 0.8112 - val_loss: 4.8914 - val_accuracy: 0.5094
Epoch 4/10
1875/1875 [==============================] - 85s 46ms/step - loss: 0.4209 - accuracy: 0.8565 - val_loss: 0.4069 - val_accuracy: 0.8519
Epoch 5/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.4237 - accuracy: 0.8539 - val_loss: 0.4478 - val_accuracy: 0.8323
Epoch 6/10
1875/1875 [==============================] - 83s 44ms/step - loss: 0.3467 - accuracy: 0.8775 - val_loss: 0.3833 - val_accuracy: 0.8562
Epoch 7/10
1875/1875 [==============================] - 84s 45ms/step - loss: 0.3846 - accuracy: 0.8703 - val_loss: 0.4579 

In [ ]:
base_model_loss, base_model_accuracy = base_model.evaluate(x_test, y_test)
print(f"Base ResNet50 Model Accuracy: {base_model_accuracy}")

313/313 [==============================] - 5s 15ms/step - loss: 0.4213 - accuracy: 0.8559
Base ResNet50 Model Accuracy: 0.85589998960495


Step 2: Design Adapters


Use ResNet50 for Feature Extraction: Freeze the layers of ResNet50 and use it to extract features from the images.

Add Custom Head with Adapters: Create a custom head with one or more adapter layers and a final classification layer.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Conv2D, ReLU, Add, Flatten

# Load the base ResNet50 model
base_model = tf.keras.applications.ResNet50(include_top=False, weights='imagenet', input_shape=(32, 32, 3))
base_model.trainable = False  # Freeze the base model

def create_adapter_module(x, reduction_factor=16):
    """Creates an adapter module."""
    # Reduce channels
    reduced = Conv2D(x.shape[-1] // reduction_factor, (1, 1), activation='relu')(x)
    # Expand channels back to original
    expanded = Conv2D(x.shape[-1], (1, 1))(reduced)
    # Add back to original input
    out = Add()([x, expanded])
    return out

# Add custom head to the base model
inputs = Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = create_adapter_module(x)  # Add adapter module
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)  # Flatten the output for the Dense layer
outputs = Dense(10, activation='softmax')(x)  # Final classification layer

# Create the complete model
model = Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 31s 13ms/step - loss: 0.8598 - accuracy: 0.6760 - val_loss: 0.6824 - val_accuracy: 0.7348
Epoch 2/10
1875/1875 [==============================] - 24s 13ms/step - loss: 0.6817 - accuracy: 0.7420 - val_loss: 0.6584 - val_accuracy: 0.7453
Epoch 3/10
1875/1875 [==============================] - 27s 14ms/step - loss: 0.6442 - accuracy: 0.7561 - val_loss: 0.6164 - val_accuracy: 0.7709
Epoch 4/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.6222 - accuracy: 0.7650 - val_loss: 0.6410 - val_accuracy: 0.7568
Epoch 5/10
1875/1875 [==============================] - 25s 13ms/step - loss: 0.6040 - accuracy: 0.7739 - val_loss: 0.5986 - val_accuracy: 0.7715
Epoch 6/10
1875/1875 [==============================] - 26s 14ms/step - loss: 0.5860 - accuracy: 0.7796 - val_loss: 0.5920 - val_accuracy: 0.7793
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.5720 - accuracy: 0.7865 - val_loss: 0.6616 -

In [ ]:
model_loss, model_accuracy = model.evaluate(x_test, y_test)
print(f"Fine tuned Model Accuracy: {model_accuracy}")

313/313 [==============================] - 4s 11ms/step - loss: 0.5972 - accuracy: 0.7758
Fine tuned Model Accuracy: 0.7757999897003174


### resnet + lora

In [ ]:
class LoRADense(tf.keras.layers.Layer):
    def __init__(self, units, rank, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.rank = rank
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        # Initialize weights to match input dimension and number of units
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=False, name='w')  # Adjusted to match input dimension
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True, name='b')

        # LoRA parameters, A is [units, rank] and B is [rank, units]
        self.lora_A = self.add_weight(shape=(self.units, self.rank), initializer='random_normal', trainable=True, name='lora_A')
        self.lora_B = self.add_weight(shape=(self.rank, input_shape[-1]), initializer='random_normal', trainable=True, name='lora_B')  # Adjusted to match input dimension

    def call(self, inputs):
        # Standard dense layer calculation with adjusted weight dimensions
        z = tf.matmul(inputs, self.w) + self.b

        # LoRA update, note the multiplication order for lora_B and inputs
        lora_update = tf.matmul(self.lora_A, tf.matmul(self.lora_B, tf.transpose(inputs))) * adaptation_rate  # Adjusted multiplication order
        lora_update = tf.transpose(lora_update)  # Transpose back to match the original dimensions

        # Apply the LoRA update to the original output
        z += lora_update

        # Apply activation function if any
        if self.activation is not None:
            z = self.activation(z)
        return z


In [ ]:
base_model = tf.keras.applications.MobileNet(input_shape=(32, 32, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the model


17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
print(base_model.output.shape)


(None, 1, 1, 1024)


In [ ]:

# Add new top layers to the base model with LoRA
inputs = Input(shape=(32, 32, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = LoRADense(128, 8, activation='relu')(x)  # Example LoRA layer
outputs = Dense(10, activation='softmax')(x)  # Final classification layer for Fashion MNIST

# Create and compile the model
model_lora = Model(inputs, outputs)
model_lora.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model_lora.fit(x_train, y_train, batch_size=32, epochs=10, validation_data=(x_test, y_test))

Epoch 1/10
1875/1875 [==============================] - 16s 7ms/step - loss: 1.8433 - accuracy: 0.3661 - val_loss: 1.6783 - val_accuracy: 0.4194
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 1.6210 - accuracy: 0.4391 - val_loss: 1.5948 - val_accuracy: 0.4566
Epoch 3/10
1875/1875 [==============================] - 13s 7ms/step - loss: 1.5631 - accuracy: 0.4590 - val_loss: 1.5527 - val_accuracy: 0.4618
Epoch 4/10
1875/1875 [==============================] - 11s 6ms/step - loss: 1.5328 - accuracy: 0.4674 - val_loss: 1.5343 - val_accuracy: 0.4661
Epoch 5/10
1875/1875 [==============================] - 12s 6ms/step - loss: 1.5126 - accuracy: 0.4745 - val_loss: 1.5192 - val_accuracy: 0.4700
Epoch 6/10
1875/1875 [==============================] - 13s 7ms/step - loss: 1.4986 - accuracy: 0.4778 - val_loss: 1.5017 - val_accuracy: 0.4785
Epoch 7/10
1875/1875 [==============================] - 15s 8ms/step - loss: 1.4873 - accuracy: 0.4807 - val_loss: 1.4954 - val_ac

In [ ]:
model_lora_loss, model_lora_accuracy = model_lora.evaluate(x_test, y_test)
print(f"Fine tuned Model with lora Accuracy: {model_lora_accuracy}")

313/313 [==============================] - 3s 8ms/step - loss: 1.4710 - accuracy: 0.4923
Fine tuned Model with lora Accuracy: 0.49230000376701355


### Semplice CNN invece di ResNet50
(funziona meglio)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

model_cnn = models.Sequential([
    # Layer 1: Convolutional layer
    layers.Conv2D(8, (3, 3), activation='relu', input_shape=(32, 32, 3)),  # Ridotto a 8 filtri
    layers.MaxPooling2D((2, 2)),

    # Flattening the 3D output to 1D
    layers.Flatten(),

    # Dense layer for classification, ridotto la dimensione a 16
    layers.Dense(16, activation='relu'),  # Ridotta la dimensione a 16

    # Output layer
    layers.Dense(10, activation='softmax')  # 10 classi per Fashion MNIST
])



In [ ]:
model_cnn.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


In [ ]:
history_model_cnn = model_cnn.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 12s 6ms/step - loss: 0.4926 - accuracy: 0.8271 - val_loss: 0.3711 - val_accuracy: 0.8691
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3317 - accuracy: 0.8827 - val_loss: 0.3437 - val_accuracy: 0.8760
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2955 - accuracy: 0.8949 - val_loss: 0.3114 - val_accuracy: 0.8893
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2721 - accuracy: 0.9041 - val_loss: 0.3153 - val_accuracy: 0.8878
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2549 - accuracy: 0.9086 - val_loss: 0.3105 - val_accuracy: 0.8909
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2390 - accuracy: 0.9143 - val_loss: 0.2973 - val_accuracy: 0.8934
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2253 - accuracy: 0.9173 - val_loss: 0.2888 - val_accuracy

In [ ]:
model_cnn_loss, model_cnn_accuracy = model_cnn.evaluate(x_test, y_test)
print(f"CNN Accuracy: {model_cnn_accuracy}")

313/313 [==============================] - 1s 3ms/step - loss: 0.3043 - accuracy: 0.8980
CNN Accuracy: 0.8980000019073486


In [ ]:
class LoRADense(tf.keras.layers.Layer):
    def __init__(self, units, rank, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.rank = rank
        self.activation = tf.keras.activations.get(activation)

    def build(self, input_shape):
        # Original Dense parameters
        self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=False, name='w')
        self.b = self.add_weight(shape=(self.units,), initializer='zeros', trainable=True, name='b')

        # LoRA parameters
        self.lora_A = self.add_weight(shape=(input_shape[-1], self.rank), initializer='random_normal', trainable=True, name='lora_A')
        self.lora_B = self.add_weight(shape=(self.rank, self.units), initializer='random_normal', trainable=True, name='lora_B')

    def call(self, inputs):
        # Standard dense layer calculation
        z = tf.matmul(inputs, self.w) + self.b

        # LoRA update
        lora_update = tf.matmul(tf.matmul(inputs, self.lora_A), self.lora_B)

        # Apply the LoRA update to the original output
        z += lora_update

        # Apply activation function if any
        if self.activation is not None:
            z = self.activation(z)
        return z

# Modifica dell'architettura del modello
model_lora2 = models.Sequential([
    layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    LoRADense(32, rank=4, activation='relu'),  # LoRA layer invece del Dense layer normale
    layers.Dense(10, activation='softmax')
])

# Compilazione e addestramento come prima
model_lora2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model_lora2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 10s 5ms/step - loss: 0.6098 - accuracy: 0.7816 - val_loss: 0.4698 - val_accuracy: 0.8252
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.4133 - accuracy: 0.8494 - val_loss: 0.4114 - val_accuracy: 0.8459
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3667 - accuracy: 0.8688 - val_loss: 0.3824 - val_accuracy: 0.8614
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3366 - accuracy: 0.8784 - val_loss: 0.3695 - val_accuracy: 0.8664
Epoch 5/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.3141 - accuracy: 0.8873 - val_loss: 0.3474 - val_accuracy: 0.8756
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2983 - accuracy: 0.8920 - val_loss: 0.3378 - val_accuracy: 0.8802
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2871 - accuracy: 0.8967 - val_loss: 0.3235 - val_accuracy

In [ ]:
model_lora2_loss, model_lora2_accuracy = model_lora2.evaluate(x_test, y_test)
print(f"CNN + Lora Accuracy: {model_lora2_accuracy}")

313/313 [==============================] - 1s 4ms/step - loss: 0.3170 - accuracy: 0.8924
CNN + Lora Accuracy: 0.8924000263214111


fine tuning selettivo:

In [ ]:
for layer in model_lora2.layers:
    layer.trainable = False


In [ ]:
# Scongela il layer LoRADense
model_lora2.layers[-2].trainable = True  # Assumendo che il LoRADense sia il penultimo layer

# Opzionale: Scongela anche l'ultimo layer Dense se vuoi affinarne i pesi
model_lora2.layers[-1].trainable = True


In [ ]:
model_lora2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model_lora2.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 40s 4ms/step - loss: 0.2478 - accuracy: 0.9098 - val_loss: 0.3093 - val_accuracy: 0.8914
Epoch 2/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2412 - accuracy: 0.9140 - val_loss: 0.3073 - val_accuracy: 0.8935
Epoch 3/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2360 - accuracy: 0.9155 - val_loss: 0.3142 - val_accuracy: 0.8942
Epoch 4/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2326 - accuracy: 0.9165 - val_loss: 0.3106 - val_accuracy: 0.8934
Epoch 5/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2284 - accuracy: 0.9183 - val_loss: 0.3086 - val_accuracy: 0.8957
Epoch 6/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2235 - accuracy: 0.9199 - val_loss: 0.3286 - val_accuracy: 0.8871
Epoch 7/10
1875/1875 [==============================] - 7s 4ms/step - loss: 0.2212 - accuracy: 0.9203 - val_loss: 0.3147 - val_accuracy

In [ ]:
model_lora2_loss, model_lora2_accuracy = model_lora2.evaluate(x_test, y_test)
print(f"CNN + Lora Accuracy: {model_lora2_accuracy}")

313/313 [==============================] - 1s 3ms/step - loss: 0.3064 - accuracy: 0.8995
CNN + Lora Accuracy: 0.8995000123977661


### Merging models (resnet + cnn(lora))

In [ ]:
model_lora2.layers

In [ ]:
base_resnet_model.layers

In [ ]:
# Per base_resnet_model, estraiamo le caratteristiche dal penultimo layer Add
resnet_features = Model(inputs=base_resnet_model.input,
                        outputs=base_resnet_model.layers[-3].output)  # Penultimo layer Add

# Per model_lora2, estraiamo le caratteristiche dal layer LoRADense
lora_features = Model(inputs=model_lora2.input,
                      outputs=model_lora2.layers[-2].output)  # Layer LoRADense


In [ ]:
from tensorflow.keras.layers import Flatten

# Input layer
input_layer = tf.keras.Input(shape=(32, 32, 3))

# Ottieni le caratteristiche dai due modelli
resnet_output = resnet_features(input_layer)
lora_output = lora_features(input_layer)

# Appiattisci l'output di ResNet per renderlo compatibile
resnet_output_flat = Flatten()(resnet_output)

# Combina le caratteristiche appiattite di ResNet con quelle di LoRADense
combined_features = Concatenate()([resnet_output_flat, lora_output])

# Aggiungi un layer denso per la classificazione finale
x = layers.Dense(256, activation='relu')(combined_features)
x = layers.Dropout(0.5)(x)  # Regolarizzazione
output_layer = layers.Dense(10, activation='softmax')(x)  # 10 classi per Fashion MNIST

# Modello combinato
combined_model = Model(inputs=input_layer, outputs=output_layer)


In [ ]:
combined_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Addestra il modello combinato sui tuoi dati Fashion MNIST
history_combined = combined_model.fit(x_train, y_train, epochs=10, validation_data=(x_test, y_test))


Epoch 1/10
1875/1875 [==============================] - 118s 47ms/step - loss: 0.4054 - accuracy: 0.8805 - val_loss: 0.3384 - val_accuracy: 0.8885
Epoch 2/10
1875/1875 [==============================] - 90s 48ms/step - loss: 0.3436 - accuracy: 0.8886 - val_loss: 0.3362 - val_accuracy: 0.8877
Epoch 3/10
1875/1875 [==============================] - 90s 48ms/step - loss: 0.3535 - accuracy: 0.8856 - val_loss: 0.3559 - val_accuracy: 0.8876
Epoch 4/10
1875/1875 [==============================] - 89s 47ms/step - loss: 0.3233 - accuracy: 0.8893 - val_loss: 0.3253 - val_accuracy: 0.8917
Epoch 5/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.3164 - accuracy: 0.8922 - val_loss: 0.3604 - val_accuracy: 0.8847
Epoch 6/10
1875/1875 [==============================] - 88s 47ms/step - loss: 0.3070 - accuracy: 0.8942 - val_loss: 0.3589 - val_accuracy: 0.8885
Epoch 7/10
1875/1875 [==============================] - 87s 46ms/step - loss: 0.3028 - accuracy: 0.8982 - val_loss: 0.3718 

In [ ]:
combined_model_loss, combined_model_accuracy = combined_model.evaluate(x_test, y_test)
print(f"Merging model Accuracy: {combined_model_accuracy}")

313/313 [==============================] - 4s 14ms/step - loss: 0.3445 - accuracy: 0.8926
Merging model Accuracy: 0.8925999999046326
